# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [49]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    #FIRST PART
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NHL']=cities['NHL'].replace(r"\[\w*\ \d*\]",'',regex=True)
    cities['NHL']=cities['NHL'].replace(r"\W",'',regex=True)
    cities['Population']=cities['Population (2016 est.)[8]']
    cities.sort_values(by=['Metropolitan area'],inplace=True)
    cities = cities[cities.NHL != ""]
    cities['Metropolitan']=cities['Metropolitan area']
    cities=cities[['Metropolitan','NHL','Population']]
    #To replace word-() ()
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
    #to replace 3 word metropolitan var (there are some that has 4 variable that why)
    #i use it twice to keep 2 word metropolitan
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    #To remove Washington(.)DC(.)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
    #We set the index
    cities.sort_values(by=['NHL'],inplace=True)
    cities.set_index('NHL',inplace=True)
    cities.shape
    
    
    #SECOND PART
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df['team']=nhl_df['team'].replace(r"\*","",regex=True)
    #We only select 2018 data - reminder that New York has 3 teams(only inside new jersey)
    nhl_df=nhl_df[nhl_df['year']==2018]
    nhl_df=nhl_df[nhl_df['GP']!=nhl_df['team']]
    nhl_df=nhl_df[['team','W','L']]
    nhl_df['NHL']=nhl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*$)",r"\2",regex=True)
    nhl_df['Metropolitan']=nhl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*$)",r"\1",regex=True)
    #nhl_df['Ratio']=pd.to_numeric(nhl_df['W'], downcast="float")/pd.to_numeric(nhl_df['L'], downcast="float")
    nhl_df['W']=pd.to_numeric(nhl_df['W'], downcast="float")
    nhl_df['L']=pd.to_numeric(nhl_df['L'], downcast="float")
    #Convert string to numeric float
    #https://www.kite.com/python/answers/how-to-convert-a-pandas-dataframe-column-of-strings-to-floats-in-python

    nhl_df=nhl_df[['Metropolitan','NHL','W','L']]
    nhl_df.sort_values(by=['Metropolitan'],inplace=True)
    nhl_df=nhl_df[['NHL','W','L']]
    nhl_df['NHL'][30]='KingsDucks'
    nhl_df['NHL'][28]='KingsDucks'
    nhl_df['NHL'][3]='MapleLeafs'
    nhl_df['NHL'][[14,17,16]]='RangersIslandersDevils'
    nhl_df['NHL'][23]='Blues'
    nhl_df['NHL'][13]='BlueJackets'
    nhl_df['NHL'][5]='RedWings'
    nhl_df['NHL'][27]='GoldenKnights'
    nhl_df.reset_index(inplace=True)
    nhl_df=nhl_df[['NHL','W','L']]
    nhl_df=nhl_df.groupby(by='NHL').mean()
    nhl_df.shape
    
    #THIRD PART
    data_t=pd.concat([cities,nhl_df],axis=1)
    data_t['Ratio']=data_t['W']/(data_t['L']+data_t['W'])
    data_t['Population']=pd.to_numeric(data_t['Population'], downcast="float")
    data_t.dtypes
    data_t
    correlation_1=data_t.corr(method ='pearson')
    correlation_1.Ratio['Population']
    
    #raise NotImplementedError()
    
    population_by_region = data_t['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data_t['Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return correlation_1.Ratio['Population']

In [50]:
type(nhl_correlation()) == np.float64

True

In [51]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


In [42]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL']=cities['NHL'].replace(r"\[\w*\ \d*\]",'',regex=True)
cities['NHL']=cities['NHL'].replace(r"\W",'',regex=True)
cities['Population']=cities['Population (2016 est.)[8]']
cities.sort_values(by=['Metropolitan area'],inplace=True)
cities = cities[cities.NHL != ""]
cities['Metropolitan']=cities['Metropolitan area']
cities=cities[['Metropolitan','NHL','Population']]
#To replace word-() ()
cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
#to replace 3 word metropolitan var (there are some that has 4 variable that why)
#i use it twice to keep 2 word metropolitan
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
#To remove Washington(.)DC(.)
cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
#We set the index
cities.sort_values(by=['NHL'],inplace=True)
cities.set_index('NHL',inplace=True)
cities

(28, 2)

In [4]:
nhl_df=pd.read_csv("assets/nhl.csv")

nhl_df['team']=nhl_df['team'].replace(r"\*","",regex=True)
#We only select 2018 data - reminder that New York has 3 teams(only inside new jersey)
nhl_df=nhl_df[nhl_df['year']==2018]
nhl_df=nhl_df[nhl_df['GP']!=nhl_df['team']]
nhl_df=nhl_df[['team','W','L']]
nhl_df['NHL']=nhl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*$)",r"\2",regex=True)
nhl_df['Metropolitan']=nhl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*$)",r"\1",regex=True)
#nhl_df['Ratio']=pd.to_numeric(nhl_df['W'], downcast="float")/pd.to_numeric(nhl_df['L'], downcast="float")
nhl_df['W']=pd.to_numeric(nhl_df['W'], downcast="float")
nhl_df['L']=pd.to_numeric(nhl_df['L'], downcast="float")
#Convert string to numeric float
#https://www.kite.com/python/answers/how-to-convert-a-pandas-dataframe-column-of-strings-to-floats-in-python

nhl_df=nhl_df[['Metropolitan','NHL','W','L']]
nhl_df.sort_values(by=['Metropolitan'],inplace=True)
nhl_df=nhl_df[['NHL','W','L']]
nhl_df['NHL'][30]='KingsDucks'
nhl_df['NHL'][28]='KingsDucks'
nhl_df['NHL'][3]='MapleLeafs'
nhl_df['NHL'][[14,17,16]]='RangersIslandersDevils'
nhl_df['NHL'][23]='Blues'
nhl_df['NHL'][13]='BlueJackets'
nhl_df['NHL'][5]='RedWings'
nhl_df['NHL'][27]='GoldenKnights'
nhl_df.reset_index(inplace=True)
nhl_df=nhl_df[['NHL','W','L']]
nhl_df=nhl_df.groupby(by='NHL').mean()
nhl_df.shape

(28, 2)

In [46]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
data_t=pd.concat([cities,nhl_df],axis=1)
data_t['Ratio']=data_t['W']/(data_t['L']+data_t['W'])
data_t['Population']=pd.to_numeric(data_t['Population'], downcast="float")
data_t.dtypes
data_t
correlation_1=data_t.corr(method ='pearson')
correlation_1.Ratio['Population']
# population_by_region = data_t['Population'] # pass in metropolitan area population from cities
# win_loss_by_region = data_t['Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
# stats.pearsonr(population_by_region, win_loss_by_region)

0.012309008105480681

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # first part
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nba_df=pd.read_csv("assets/nba.csv")
    nba_df=nba_df[nba_df['year']==2018]
    nba_df=nba_df[['team','W','L']]
    nba_df=nba_df[nba_df['team']!=nba_df['W']]
    # nba_df['team']=nba_df['team'].replace(r"(^\w*\ \w*\ \w*|^\w*\ \w*)(\*\ \(\d*\)|\ \(\d*\))",r"\1",regex=True)
    nba_df['team']=nba_df['team'].replace(r"\*","",regex=True)
    nba_df['team']=nba_df['team'].replace(r"\(\d*\)","",regex=True)

    nba_df['NBA']=nba_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\2",regex=True)
    nba_df['Metropolitan']=nba_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\1",regex=True)
    nba_df=nba_df[['Metropolitan','NBA','W','L']]
    nba_df.sort_values(by=['NBA'],inplace=True)
    nba_df.set_index('NBA')
    nba_df['NBA'][17]='TrailBlazers'
    nba_df['NBA'][11,10]="KnicksNets"
    nba_df['NBA'][25,24]="LakersClippers"
    nba_df['W']=pd.to_numeric(nba_df['W'], downcast="float")
    nba_df['L']=pd.to_numeric(nba_df['L'], downcast="float")
    nba_df=nba_df.groupby(by='NBA').mean()
    nba_df.reset_index(inplace=True)
    nba_df
    
    # Second part
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NBA']=cities['NBA'].replace(r"\[\w*\ \d*\]",'',regex=True)
    cities['NBA']=cities['NBA'].replace(r"\W",'',regex=True)
    cities['Population']=cities['Population (2016 est.)[8]']
    cities.sort_values(by=['Metropolitan area'],inplace=True)
    cities = cities[cities.NBA != ""]
    cities['Metropolitan']=cities['Metropolitan area']
    cities=cities[['Metropolitan','NBA','Population']]
    #To replace word-() ()
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
    #to replace 3 word metropolitan var (there are some that has 4 variable that why)
    #i use it twice to keep 2 word metropolitan
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    #To remove Washington(.)DC(.)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
    #We set the index
    cities.sort_values(by=['NBA'],inplace=True)
    cities_3=cities[['NBA','Population']]
    cities_3.set_index('NBA',inplace=True)
    cities_3.reset_index(inplace=True)
    cities_3['NBA_1']=cities_3['NBA']
    cities_3
    
    #third part
    data_f=pd.concat([cities_3,nba_df],axis=1)
    data_f['Population']=pd.to_numeric(data_f['Population'], downcast="float")
    data_f['Ratio']=data_f['W']/(data_f['W']+data_f['L'])
    data_f=data_f[['NBA_1','Population','Ratio']]
    data_f.set_index('NBA_1',inplace=True)
    data_f
    correlation_2=data_f.corr(method ='pearson')
    correlation_2.Ratio['Population']
    
    #raise NotImplementedError()
    
    population_by_region = data_f['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data_f['Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return correlation_2.Ratio['Population']

In [38]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NBA']=cities['NBA'].replace(r"\[\w*\ \d*\]",'',regex=True)
cities['NBA']=cities['NBA'].replace(r"\W",'',regex=True)
cities['Population']=cities['Population (2016 est.)[8]']
cities.sort_values(by=['Metropolitan area'],inplace=True)
cities = cities[cities.NBA != ""]
cities['Metropolitan']=cities['Metropolitan area']
cities=cities[['Metropolitan','NBA','Population']]
#To replace word-() ()
cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
#to replace 3 word metropolitan var (there are some that has 4 variable that why)
#i use it twice to keep 2 word metropolitan
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
#To remove Washington(.)DC(.)
cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
#We set the index
cities.sort_values(by=['NBA'],inplace=True)
cities_3=cities[['NBA','Population']]
cities_3.set_index('NBA',inplace=True)
cities_3.reset_index(inplace=True)
cities_3['NBA_1']=cities_3['NBA']
cities_3

,NBA,Population,NBA_1
0,76ers,6070500,76ers
1,Bucks,1572482,Bucks
2,Bulls,9512999,Bulls
3,Cavaliers,2055612,Cavaliers
4,Celtics,4794447,Celtics
5,Grizzlies,1342842,Grizzlies
6,Hawks,5789700,Hawks
7,Heat,6066387,Heat
8,Hornets,2474314,Hornets
9,Jazz,1186187,Jazz


In [39]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
nba_df=nba_df[nba_df['year']==2018]
nba_df=nba_df[['team','W','L']]
nba_df=nba_df[nba_df['team']!=nba_df['W']]
# nba_df['team']=nba_df['team'].replace(r"(^\w*\ \w*\ \w*|^\w*\ \w*)(\*\ \(\d*\)|\ \(\d*\))",r"\1",regex=True)
nba_df['team']=nba_df['team'].replace(r"\*","",regex=True)
nba_df['team']=nba_df['team'].replace(r"\(\d*\)","",regex=True)

nba_df['NBA']=nba_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\2",regex=True)
nba_df['Metropolitan']=nba_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\1",regex=True)
nba_df=nba_df[['Metropolitan','NBA','W','L']]
nba_df.sort_values(by=['NBA'],inplace=True)
nba_df.set_index('NBA')
nba_df['NBA'][17]='TrailBlazers'
nba_df['NBA'][11,10]="KnicksNets"
nba_df['NBA'][25,24]="LakersClippers"
nba_df['W']=pd.to_numeric(nba_df['W'], downcast="float")
nba_df['L']=pd.to_numeric(nba_df['L'], downcast="float")
nba_df=nba_df.groupby(by='NBA').mean()
nba_df.reset_index(inplace=True)
nba_df

,NBA,W,L
0,76ers,52.0,30.0
1,Bucks,44.0,38.0
2,Bulls,27.0,55.0
3,Cavaliers,50.0,32.0
4,Celtics,55.0,27.0
5,Grizzlies,22.0,60.0
6,Hawks,24.0,58.0
7,Heat,44.0,38.0
8,Hornets,36.0,46.0
9,Jazz,48.0,34.0


In [53]:
data_f=pd.concat([cities_3,nba_df],axis=1)
data_f['Population']=pd.to_numeric(data_f['Population'], downcast="float")
data_f['Ratio']=data_f['W']/(data_f['W']+data_f['L'])
data_f=data_f[['NBA_1','Population','Ratio']]
data_f.set_index('NBA_1',inplace=True)
data_f
correlation_2=data_f.corr(method ='pearson')
correlation_2    
#correlation_2.Ratio['Population']

,Population,Ratio
Population,1.000000,-0.176572
Ratio,-0.176572,1.000000


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    #First part
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df=mlb_df[mlb_df['year']==2018]
    mlb_df=mlb_df[['team','W','L']]
    mlb_df=mlb_df[mlb_df['team']!=mlb_df['W']]
    mlb_df
    # mlb_df['team']=mlb_df['team'].replace(r"(^\w*\ \w*\ \w*|^\w*\ \w*)(\*\ \(\d*\)|\ \(\d*\))",r"\1",regex=True)
    mlb_df['team']=mlb_df['team'].replace(r"\*","",regex=True)
    mlb_df['team']=mlb_df['team'].replace(r"\(\d*\)","",regex=True)

    mlb_df['MLB']=mlb_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\2",regex=True)
    mlb_df['Metropolitan']=mlb_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\1",regex=True)
    mlb_df=mlb_df[['Metropolitan','MLB','W','L']]
    mlb_df.sort_values(by=['MLB'],inplace=True)
    mlb_df.set_index('MLB')
    mlb_df
    mlb_df['MLB'][13,25]='DodgersAngels'
    mlb_df['MLB'][28,11]='GiantsAthletics'
    mlb_df['MLB'][21,8]='CubsWhiteSox'
    mlb_df['MLB'][3]='BlueJays'
    mlb_df['MLB'][0]='RedSox'
    mlb_df['MLB'][22]='Cardinals'
    mlb_df['MLB'][1,18]='YankeesMets'
    mlb_df.sort_values(by=['MLB'],inplace=True)
    mlb_df['W']=pd.to_numeric(mlb_df['W'], downcast="float")
    mlb_df['L']=pd.to_numeric(mlb_df['L'], downcast="float")
    mlb_df=mlb_df.groupby(by='MLB').mean()
    mlb_df.reset_index(inplace=True)
    mlb_df
    
    #Second part
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities
    cities['MLB']=cities['MLB'].replace(r"\[\w*\ \d*\]",'',regex=True)
    cities['MLB']=cities['MLB'].replace(r"\W",'',regex=True)
    cities['Population']=cities['Population (2016 est.)[8]']
    cities.sort_values(by=['Metropolitan area'],inplace=True)
    cities = cities[cities.MLB != ""]
    cities['Metropolitan']=cities['Metropolitan area']
    cities=cities[['Metropolitan','MLB','Population']]
    #To replace word-() ()
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
    #to replace 3 word metropolitan var (there are some that has 4 variable that why)
    #i use it twice to keep 2 word metropolitan
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    #To remove Washington(.)DC(.)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
    #We set the index
    cities.sort_values(by=['MLB'],inplace=True)
    cities_4=cities[['MLB','Population']]
    cities_4.set_index('MLB',inplace=True)
    cities_4.reset_index(inplace=True)
    cities_4['MLB_1']=cities_4['MLB']
    cities_4
    
    #Third part
    data_r=pd.concat([cities_4,mlb_df],axis=1)
    data_r['Population']=pd.to_numeric(data_r['Population'], downcast="float")
    data_r['Ratio']=data_r['W']/(data_r['W']+data_r['L'])
    data_r=data_r[['MLB_1','Population','Ratio']]
    data_r.set_index('MLB_1',inplace=True)
    
    correlation_3=data_r.corr(method ='pearson')
    correlation_3.Ratio['Population']
    
    #raise NotImplementedError()
    
    population_by_region = data_r['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data_r['Ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return correlation_3.Ratio['Population']

In [13]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities
cities['MLB']=cities['MLB'].replace(r"\[\w*\ \d*\]",'',regex=True)
cities['MLB']=cities['MLB'].replace(r"\W",'',regex=True)
cities['Population']=cities['Population (2016 est.)[8]']
cities.sort_values(by=['Metropolitan area'],inplace=True)
cities = cities[cities.MLB != ""]
cities['Metropolitan']=cities['Metropolitan area']
cities=cities[['Metropolitan','MLB','Population']]
#To replace word-() ()
cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
#to replace 3 word metropolitan var (there are some that has 4 variable that why)
#i use it twice to keep 2 word metropolitan
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
#To remove Washington(.)DC(.)
cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
#We set the index
cities.sort_values(by=['MLB'],inplace=True)
cities_4=cities[['MLB','Population']]
cities_4.set_index('MLB',inplace=True)
cities_4.reset_index(inplace=True)
cities_4['MLB_1']=cities_4['MLB']
cities_4.shape

(26, 3)

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df=mlb_df[mlb_df['year']==2018]
mlb_df=mlb_df[['team','W','L']]
mlb_df=mlb_df[mlb_df['team']!=mlb_df['W']]
mlb_df
# mlb_df['team']=mlb_df['team'].replace(r"(^\w*\ \w*\ \w*|^\w*\ \w*)(\*\ \(\d*\)|\ \(\d*\))",r"\1",regex=True)
mlb_df['team']=mlb_df['team'].replace(r"\*","",regex=True)
mlb_df['team']=mlb_df['team'].replace(r"\(\d*\)","",regex=True)

mlb_df['MLB']=mlb_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\2",regex=True)
mlb_df['Metropolitan']=mlb_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\1",regex=True)
mlb_df=mlb_df[['Metropolitan','MLB','W','L']]
mlb_df.sort_values(by=['MLB'],inplace=True)
mlb_df.set_index('MLB')
mlb_df
mlb_df['MLB'][13,25]='DodgersAngels'
mlb_df['MLB'][28,11]='GiantsAthletics'
mlb_df['MLB'][21,8]='CubsWhiteSox'
mlb_df['MLB'][3]='BlueJays'
mlb_df['MLB'][0]='RedSox'
mlb_df['MLB'][22]='Cardinals'
mlb_df['MLB'][1,18]='YankeesMets'
mlb_df.sort_values(by=['MLB'],inplace=True)
mlb_df['W']=pd.to_numeric(mlb_df['W'], downcast="float")
mlb_df['L']=pd.to_numeric(mlb_df['L'], downcast="float")
mlb_df=mlb_df.groupby(by='MLB').mean()
mlb_df.reset_index(inplace=True)
mlb_df.shape

(26, 3)

In [12]:
data_r=pd.concat([cities_4,mlb_df],axis=1)
data_r['Population']=pd.to_numeric(data_r['Population'], downcast="float")
data_r['Ratio']=data_r['W']/(data_r['W']+data_r['L'])
data_r=data_r[['MLB_1','Population','Ratio']]
data_r.set_index('MLB_1',inplace=True)
    
correlation_3=data_r.corr(method ='pearson')
correlation_3.Ratio['Population']

0.15726078612736338

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [45]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities
    cities['NFL']=cities['NFL'].replace(r"\[\w*\ \d*\]",'',regex=True)
    cities['NFL']=cities['NFL'].replace(r"\W",'',regex=True)
    cities['Population']=cities['Population (2016 est.)[8]']
    cities.sort_values(by=['Metropolitan area'],inplace=True)
    cities = cities[cities.NFL != ""]
    cities['Metropolitan']=cities['Metropolitan area']
    cities=cities[['Metropolitan','NFL','Population']]
    #To replace word-() ()
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
    #to replace 3 word metropolitan var (there are some that has 4 variable that why)
    #i use it twice to keep 2 word metropolitan
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
    #To remove Washington(.)DC(.)
    cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
    #We set the index
    cities.sort_values(by=['NFL'],inplace=True)
    cities_5=cities[['NFL','Population']]
    cities_5.set_index('NFL',inplace=True)
    cities_5.reset_index(inplace=True)
    cities_5['NFL_1']=cities_5['NFL']
    cities_5
    # cities

    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df=nfl_df[nfl_df['year']==2018]
    nfl_df=nfl_df[['team','W','L']]
    nfl_df=nfl_df[nfl_df['team']!=nfl_df['W']]
    nfl_df['team']=nfl_df['team'].replace(r"\*","",regex=True)
    nfl_df['team']=nfl_df['team'].replace(r"\(\d*\)","",regex=True)
    nfl_df
    nfl_df['NFL']=nfl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\2",regex=True)
    nfl_df['Metropolitan']=nfl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\1",regex=True)
    nfl_df=nfl_df[['Metropolitan','NFL','W','L']]
    nfl_df.sort_values(by=['NFL'],inplace=True)
    nfl_df.set_index('NFL')
    nfl_df

    nfl_df['NFL'][38,19]='49ersRaiders'
    nfl_df['NFL'][17,36]='RamsChargers'
    nfl_df['NFL'][12]='Colts'
    nfl_df['NFL'][22]='Eagles'
    nfl_df['NFL'][24,4]='GiantsJets'
    nfl_df['NFL'][37]='Seahawks'

    nfl_df.sort_values(by=['NFL'],inplace=True)
    nfl_df['W']=pd.to_numeric(nfl_df['W'], downcast="float")
    nfl_df['L']=pd.to_numeric(nfl_df['L'], downcast="float")
    nfl_df=nfl_df.groupby(by='NFL').mean()
    nfl_df.reset_index(inplace=True)
    nfl_df    
    
    data_g=pd.concat([cities_5,nfl_df],axis=1)
    data_g['Population']=pd.to_numeric(data_g['Population'], downcast="float")
    data_g['Ratio']=data_g['W']/(data_g['W']+data_g['L'])
    data_g=data_g[['NFL_1','Population','Ratio']]
    data_g.set_index('NFL_1',inplace=True)

    correlation_4=data_g.corr(method ='pearson')
    correlation_4.Ratio['Population']
    #raise NotImplementedError()
    
    population_by_region = data_g['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data_g['Ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return correlation_4.Ratio['Population']

In [44]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities
cities['NFL']=cities['NFL'].replace(r"\[\w*\ \d*\]",'',regex=True)
cities['NFL']=cities['NFL'].replace(r"\W",'',regex=True)
cities['Population']=cities['Population (2016 est.)[8]']
cities.sort_values(by=['Metropolitan area'],inplace=True)
cities = cities[cities.NFL != ""]
cities['Metropolitan']=cities['Metropolitan area']
cities=cities[['Metropolitan','NFL','Population']]
#To replace word-() ()
cities['Metropolitan']=cities['Metropolitan'].replace(r"\–\w+\ \w*$",'',regex=True)
#to replace 3 word metropolitan var (there are some that has 4 variable that why)
#i use it twice to keep 2 word metropolitan
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
cities['Metropolitan']=cities['Metropolitan'].replace(r"(\w*\ \w*)(\ \w*)",r"\1",regex=True)
#To remove Washington(.)DC(.)
cities['Metropolitan']=cities['Metropolitan'].replace(r"\,\ \w\.\w\.","",regex=True)
#We set the index
cities.sort_values(by=['NFL'],inplace=True)
cities_5=cities[['NFL','Population']]
cities_5.set_index('NFL',inplace=True)
cities_5.reset_index(inplace=True)
cities_5['NFL_1']=cities_5['NFL']
cities_5
# cities

,NFL,Population,NFL_1
0,49ersRaiders,6657982,49ersRaiders
1,Bears,9512999,Bears
2,Bengals,2165139,Bengals
3,Bills,1132804,Bills
4,Broncos,2853077,Broncos
5,Browns,2055612,Browns
6,Buccaneers,3032171,Buccaneers
7,Cardinals,4661537,Cardinals
8,Chiefs,2104509,Chiefs
9,Colts,2004230,Colts


In [34]:
nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df=nfl_df[nfl_df['year']==2018]
nfl_df=nfl_df[['team','W','L']]
nfl_df=nfl_df[nfl_df['team']!=nfl_df['W']]
nfl_df['team']=nfl_df['team'].replace(r"\*","",regex=True)
nfl_df['team']=nfl_df['team'].replace(r"\(\d*\)","",regex=True)
nfl_df
nfl_df['NFL']=nfl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\2",regex=True)
nfl_df['Metropolitan']=nfl_df['team'].replace(r"(\w*\ \w*\ |\w*\ )(\w*)",r"\1",regex=True)
nfl_df=nfl_df[['Metropolitan','NFL','W','L']]
nfl_df.sort_values(by=['NFL'],inplace=True)
nfl_df.set_index('NFL')
nfl_df

nfl_df['NFL'][38,19]='49ersRaiders'
nfl_df['NFL'][17,36]='RamsChargers'
nfl_df['NFL'][12]='Colts'
nfl_df['NFL'][22]='Eagles'
nfl_df['NFL'][24,4]='GiantsJets'
nfl_df['NFL'][37]='Seahawks'

nfl_df.sort_values(by=['NFL'],inplace=True)
nfl_df['W']=pd.to_numeric(nfl_df['W'], downcast="float")
nfl_df['L']=pd.to_numeric(nfl_df['L'], downcast="float")
nfl_df=nfl_df.groupby(by='NFL').mean()
nfl_df.reset_index(inplace=True)
nfl_df

,NFL,W,L
0,49ersRaiders,4.0,12.0
1,Bears,12.0,4.0
2,Bengals,6.0,10.0
3,Bills,6.0,10.0
4,Broncos,6.0,10.0
5,Browns,7.0,8.0
6,Buccaneers,5.0,11.0
7,Cardinals,3.0,13.0
8,Chiefs,12.0,4.0
9,Colts,10.0,6.0


In [36]:
data_g=pd.concat([cities_5,nfl_df],axis=1)
data_g['Population']=pd.to_numeric(data_g['Population'], downcast="float")
data_g['Ratio']=data_g['W']/(data_g['W']+data_g['L'])
data_g=data_g[['NFL_1','Population','Ratio']]
data_g.set_index('NFL_1',inplace=True)
    
correlation_4=data_g.corr(method ='pearson')
correlation_4.Ratio['Population']

0.004922106003065314

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values